In [1]:
# importing all necessary libraries 
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
# defing 

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils #Draw utilities 

In [3]:
# function for detecting pose of the person using MideaPipe

def multiple_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # Color convertion 
    image.flags.writeable = False
    results = model.process(image)                # Precessing the image via holistic model to generate pose keypoints
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # Color convertion again
    return image,results                          # returning generated pose keypoints as results and image 

In [4]:

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:


def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [7]:
# define data_path where your all extracted body pose keypoints files are saved frame-wise in .npy format
DATA_PATH=r"D:\Cricket_images"

In [8]:
actions=os.listdir(DATA_PATH)
actions

['Cover Drive', 'Helicopter', 'Late Cut', 'Pull', 'Straight Drive', 'Sweep']

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map 

{'Cover Drive': 0,
 'Helicopter': 1,
 'Late Cut': 2,
 'Pull': 3,
 'Straight Drive': 4,
 'Sweep': 5}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))):
        window = []
        res = np.load(os.path.join(DATA_PATH, action, str(sequence)))
        window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
sequences[0]

[array([0., 0., 0., ..., 0., 0., 0.])]

In [13]:
labels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
np.array(sequences).shape

(3600, 1, 1662)

In [15]:
np.array(labels).shape

(3600,)

In [16]:
X = np.array(sequences)

In [17]:
from tensorflow.keras.utils import to_categorical

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05,shuffle=True)

In [22]:
print(X_train.shape)
print(X_test.shape)

(3420, 1, 1662)
(180, 1, 1662)


In [23]:
print(y_train.shape)
print(y_test.shape)

(3420, 6)
(180, 6)


In [24]:
len(actions)

6

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback


In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
model.fit(X_train, y_train, epochs=50,batch_size=128,shuffle = True,validation_split = 0.20)

Epoch 1/50
22/22 [==============================] - 1s 23ms/step - loss: 0.1126 - accuracy: 0.9518 - val_loss: 0.3031 - val_accuracy: 0.9196
Epoch 2/50
22/22 [==============================] - 0s 18ms/step - loss: 0.1123 - accuracy: 0.9518 - val_loss: 0.3033 - val_accuracy: 0.9196
Epoch 3/50
22/22 [==============================] - 0s 19ms/step - loss: 0.1124 - accuracy: 0.9518 - val_loss: 0.3041 - val_accuracy: 0.9167
Epoch 4/50
22/22 [==============================] - 0s 20ms/step - loss: 0.1123 - accuracy: 0.9518 - val_loss: 0.3017 - val_accuracy: 0.9211
Epoch 5/50
22/22 [==============================] - 0s 20ms/step - loss: 0.1123 - accuracy: 0.9518 - val_loss: 0.3035 - val_accuracy: 0.9196
Epoch 6/50
22/22 [==============================] - 0s 21ms/step - loss: 0.1124 - accuracy: 0.9518 - val_loss: 0.3030 - val_accuracy: 0.9196
Epoch 7/50
22/22 [==============================] - 0s 20ms/step - loss: 0.1123 - accuracy: 0.9518 - val_loss: 0.3043 - val_accuracy: 0.9196
Epoch 8/50
22

In [76]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             442112    
                                                                 
 lstm_1 (LSTM)               (None, 1, 128)            98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-trai

In [77]:
res = model.predict(X_test)

6/6 [==============================] - 0s 5ms/step


In [78]:
res.shape

(180, 6)

In [79]:
res[:5]

array([[7.4648547e-19, 4.7833406e-16, 9.9999988e-01, 1.2892329e-11,
        9.6054409e-10, 1.2109030e-07],
       [1.2161336e-19, 6.1333714e-28, 4.7579020e-30, 1.1094484e-22,
        3.8185805e-10, 1.0000000e+00],
       [1.4085560e-03, 9.9858958e-01, 1.8065070e-06, 3.4646747e-11,
        3.5386576e-09, 3.1518493e-11],
       [3.2127216e-01, 3.6163965e-01, 1.4213195e-01, 4.4351902e-02,
        4.2903461e-02, 8.7700777e-02],
       [1.8547825e-11, 1.0000000e+00, 6.1514690e-14, 3.0240974e-16,
        3.6640358e-22, 2.4954952e-20]], dtype=float32)

In [80]:
pred=[np.argmax(i) for i in res]

In [81]:
pred[:10]

[2, 5, 1, 1, 1, 5, 4, 3, 0, 3]

In [82]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [83]:
actual=[np.argmax(i) for i in y_test]

In [84]:
actual[:10]

[2, 5, 1, 1, 1, 5, 4, 3, 0, 3]

In [85]:
multilabel_confusion_matrix(actual, pred)

array([[[147,   1],
        [  1,  31]],

       [[146,   4],
        [  0,  30]],

       [[148,   1],
        [  5,  26]],

       [[144,   0],
        [  1,  35]],

       [[154,   2],
        [  4,  20]],

       [[148,   5],
        [  2,  25]]], dtype=int64)

In [86]:
accuracy_score(actual,pred)

0.9277777777777778

In [87]:
model.save('cricket_pose_detection.h5')